In [72]:
import cv2 as cv
import pyrealsense2 as rs2
import open3d as o3d
import numpy as np
import time

from spygrt.stream import Recording, NoMoreFrames, DualRecording
from spygrt.tracker import Tracker
#rec1 = Recording('C:/Users/yben4607/clean/ResearchData/VISION/Youssef_Test/Recording_Session/121622061907_2022May24015518.bag')
#rec2 = Recording('C:/Users/yben4607/clean/ResearchData/VISION/Youssef_Test/Recording_Session/123122061443_2022May24015518.bag')
#rec1.load_calibration('C:/Users/yben4607/clean/ResearchData/VISION/Youssef_Test/RS_Calibration/121622061907_cal.txt')
#rec2.load_calibration('C:/Users/yben4607/clean/ResearchData/VISION/Youssef_Test/RS_Calibration/123122061443_cal.txt')
rec1 = Recording('E:/VISION/133/Recording_Session/121622061907_2023Mar23023414.bag')
rec2 = Recording('E:/VISION/129/Recording_Session/123122061443_2023Mar23023414.bag')
rec1.load_calibration('121622061907_opti.txt')
rec2.load_calibration('123122061443_opti.txt')




array([[ 0.76596981,  0.47690493, -0.46312463,  0.40999882],
       [ 0.65983385, -0.45945311,  0.61818653, -0.47658443],
       [ 0.0808824 , -0.76817775, -0.65726215,  0.49703872],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [73]:
d = DualRecording(rec1,rec2)

In [74]:
d.stream1._playback.current_status()

<playback_status.stopped: 3>

In [75]:
d.start_stream()

In [76]:
d.get_frames()

((<pyrealsense2.frame Z16 #1160 @1676258444174.644043>,
  <pyrealsense2.frame RGB8 #1194 @1676258444174.980957>),
 (<pyrealsense2.frame Z16 #1121 @1676258444195.972412>,
  <pyrealsense2.frame RGB8 #1123 @1676258444196.309326>))

In [77]:
t = Tracker(ref_surface=d.pcd)

[Open3D INFO] Picked point #224438 (-0.12, 0.067, 0.15) to add in queue.
[Open3D INFO] Picked point #1162817 (0.018, -0.047, 0.16) to add in queue.


In [2]:

t = Tracker(ref_surface=d.pcd)


[Open3D INFO] Picked point #195005 (0.089, -0.065, 0.16) to add in queue.
[Open3D INFO] Picked point #189889 (0.087, -0.062, 0.17) to add in queue.
[Open3D INFO] Remove picked point #189889 from pick queue.
[Open3D INFO] Remove picked point #195005 from pick queue.
[Open3D INFO] Picked point #209075 (0.095, -0.071, 0.15) to add in queue.
[Open3D INFO] Picked point #1162800 (-0.031, 0.056, 0.16) to add in queue.


In [3]:
o3d.visualization.draw_geometries([t.ref_surface.to_legacy()])

In [78]:
from datetime import timedelta
delta = timedelta(seconds=600)
d.stream1.seek(delta)
d.stream2.seek(delta)

In [7]:
p1 = [0.3,0.3,0.3]
p2 = [-0.3,-0.3,0.2]
midz = np.mean([p1[2], p2[2]])
lower = [np.min([p1[0], p2[0]]), np.min([p1[1], p2[1]]), midz - 0.02]

In [8]:
 lower

[-0.3, -0.3, 0.23]

In [10]:
type(d.pcd.point['positions'].numpy()[0][0])

numpy.float32

In [8]:
t.get_motion(d.compute_pcd(True))

[[0.999996 -0.00115655 0.00258873 -0.000497843],
 [0.00115808 0.999999 -0.000587933 0.000163875],
 [-0.00258804 0.000590929 0.999996 2.50675e-06],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CUDA:0, 0x130a3b6000]

In [7]:
a=d.compute_pcd(True)

In [10]:
a is None

True

In [14]:
d.get_frames()

((<pyrealsense2.frame Z16 #8056 @1653364518745.431152>,
  <pyrealsense2.frame RGB8 #8052 @1653364518728.857178>),
 (<pyrealsense2.frame Z16 #8013 @1653364518748.400391>,
  <pyrealsense2.frame RGB8 #8008 @1653364518732.104736>))

In [15]:
d.pose is None

False

In [18]:
d.stream1.compute_pcd() + d.stream2.compute_pcd()

PointCloud on CPU:0 [1843200 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {1843200, 3}).

In [4]:
t.ref_surface.estimate_normals(30,0.001)

In [8]:
Tracker.compute_rotation(t._t_guess)

AttributeError: type object 'Tracker' has no attribute 'isR_matrix'

In [9]:
data = []
import time
a = time.time()
mat_arr = []
data.append(np.concatenate(([d.timestamp], Tracker.compute_translation(t.t_guess), Tracker.compute_rotation(t.t_guess))))
mat_arr.append(t._t_guess)
for i in range(120000):
    
    try:
        t.get_motion(d.compute_pcd(True))
    except RuntimeError:
        continue
    except NoMoreFrames:
        np.savetxt('track.txt',data)
        break
    data.append(np.concatenate(([d.timestamp], Tracker.compute_translation(t.t_guess), Tracker.compute_rotation(t.t_guess))))
    mat_arr.append(t._t_guess)
    if i%100 == 0:
        print(i/100)
        np.savetxt('track.txt',data)
b = time.time()
b-a   

0.0
[Open3D WARNING] 0 correspondence present between the pointclouds. Try increasing the max_correspondence_distance parameter.
1.0
Timestamps:1676258453347.0708 1433 1676258453469.0671 1397
Timestamps:1676258454422.1592 1465 1676258454544.2842 1429
Timestamps:1676258454422.1592 1465 1676258454544.2842 1429
Timestamps:1676258455194.96 1488 1676258455317.0886 1452
Timestamps:1676258457782.2197 1565 1676258457904.2498 1529
2.0
Timestamps:1676258458823.7617 1596 1676258458945.8157 1560
Timestamps:1676258461713.4575 1682 1676258461835.3643 1646
3.0
Timestamps:1676258467996.565 1869 1676258468118.3394 1833
4.0
Timestamps:1676258473540.3994 2034 1676258473662.154 1998
Timestamps:1676258475455.641 2091 1676258475577.3076 2055
Timestamps:1676258475455.641 2091 1676258475577.3076 2055
5.0
Timestamps:1676258479756.5095 2219 1676258479642.7573 2176
Timestamps:1676258479857.305 2222 1676258479743.5542 2179
Timestamps:1676258480327.6821 2236 1676258480213.9392 2193
Timestamps:1676258480327.6821 22

39.0
Timestamps:1676258716566.897 9267 1676258716716.7876 9232
Timestamps:1676258717339.7031 9290 1676258717455.983 9254
40.0
41.0
Timestamps:1676258727150.6404 9582 1676258727267.0044 9546
Timestamps:1676258727150.6404 9582 1676258727267.0044 9546
Timestamps:1676258732526.5764 9742 1676258732407.6956 9699
Timestamps:1676258733803.3887 9780 1676258733650.84 9736
42.0
Timestamps:1676258737700.954 9896 1676258737816.997 9860
43.0
Timestamps:1676258747310.4092 10182 1676258747460.0625 10147
44.0
Timestamps:1676258750502.4324 10277 1676258750618.243 10241
Timestamps:1676258750636.826 10281 1676258750786.2268 10246
45.0
Timestamps:1676258754400.022 10393 1676258754515.703 10357
Timestamps:1676258754904.001 10408 1676258754784.5186 10365
Timestamps:1676258756147.17 10445 1676258756027.7017 10402
Timestamps:1676258757793.5222 10494 1676258757673.9614 10451
46.0
Timestamps:1676258763740.6443 10671 1676258763856.39 10635
Timestamps:1676258764647.8308 10698 1676258764763.5452 10662
Timestamps:16

79.0
Timestamps:1676258988522.4001 17361 1676258988633.221 17325
Timestamps:1676258988522.4001 17361 1676258988633.221 17325
Timestamps:1676258992083.9368 17467 1676258991959.3916 17424
Timestamps:1676258992083.9368 17467 1676258991959.3916 17424
80.0
Timestamps:1676258996351.2114 17594 1676258996461.7917 17558
Timestamps:1676258997896.841 17640 1676258998007.2366 17604
Timestamps:1676258997896.841 17640 1676258998007.2366 17604
Timestamps:1676259000248.735 17710 1676259000392.7705 17675
Timestamps:1676259000483.945 17717 1676259000594.3506 17681
81.0
Timestamps:1676259006632.7188 17900 1676259006742.9602 17864
Timestamps:1676259006632.7188 17900 1676259006742.9602 17864
82.0
Timestamps:1676259010530.1333 18016 1676259010674.0103 17981
Timestamps:1676259012042.102 18061 1676259012152.2864 18025
Timestamps:1676259012042.102 18061 1676259012152.2864 18025
Timestamps:1676259012613.3015 18078 1676259012488.2607 18035
Timestamps:1676259012814.9036 18084 1676259012958.6536 18049
Timestamps:1

4364.039617538452

In [4]:
o3d.visualization.draw_geometries([t._source.to_legacy()])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [8]:
t.t_guess

[[0.960616 -0.26193 -0.0927886 0.0517465],
 [0.276662 0.932738 0.23121 -0.0737735],
 [0.0259866 -0.247775 0.968469 0.0439792],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CPU:0, 0x24d64541970]

In [7]:
saved_data = data

In [17]:
for i in range(48):
    d.compute_pcd(True)

In [18]:
t.get_motion(d.pcd)

[[0.976428 -0.135231 0.168226 -0.0104739],
 [0.146373 0.987664 -0.0556359 -0.0120752],
 [-0.158627 0.0789482 0.984177 -0.068689],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CPU:0, 0x18e6fd1f110]

In [10]:
o3d.visualization.draw_geometries([d.pcd.to_legacy()])

In [3]:
np.savetxt('data.txt',np.asarray(data))

In [5]:
d.timestamp

1676258858178.8535

In [4]:
len(data)

496

In [7]:
a=1.676258596709123047e+12
b=1.676258444235704590e+12
a-b

152473.41845703125

In [5]:
import numpy as np
R = np.asarray([[1,2,3,4],[5,6,7,8],[9, 10 ,11, 12],[13,14,15,16]])

In [6]:
R

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [7]:
R[0:3].T[0:3].T

array([[ 1,  2,  3],
       [ 5,  6,  7],
       [ 9, 10, 11]])

In [9]:
(1676258858178.8535-a)/1000/60

4.3578288411458335

In [10]:
np.savetxt('track.txt',data)

In [10]:
a = o3d.core.Tensor(np.asarray([]),device=o3d.core.Device('cuda:0'))

In [11]:
a.is_cuda

True

In [7]:
1.676259139169756836e+12-1.676258444218906494e+12

694950.8503417969

In [6]:
I = np.asarray([[1,0,0,0.022],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
rec1._pose = I@rec1._pose
rec2._pose = I@rec2._pose

In [7]:
d.compute_pcd(False)

PointCloud on CUDA:0 [1843200 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {1843200, 3}).

In [9]:
data = []
data.append(np.concatenate(([d.timestamp], Tracker.compute_translation(t.t_guess), Tracker.compute_rotation(t.t_guess))))


In [10]:
data

[array([ 1.67625844e+12, -4.97843153e-04,  1.63874577e-04,  2.50674742e-06,
         6.63530665e-02,  1.48284196e-01,  3.38578251e-02])]

In [14]:
np.savetxt('track.txt',data)

In [17]:
np.savetxt(rec2.serial + 'opti.txt',rec2.pose)

In [13]:
o3d.io.write_point_cloud('a.pcd', d.pcd.to_legacy())

True

In [13]:
t.t_guess

[[0.999996 -0.00115655 0.00258873 -0.000497843],
 [0.00115808 0.999999 -0.000587933 0.000163875],
 [-0.00258804 0.000590929 0.999996 2.50675e-06],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CUDA:0, 0x130a3b6000]

In [2]:
Tracker.compute_translation("a'")

AttributeError: 'str' object has no attribute 'is_cuda'

In [19]:
print(mat_arr[-1])
print(data[-1])

[[0.999986 0.00115013 0.00516536 -0.000762900],
 [-0.00108192 0.999912 -0.0131894 0.00201227],
 [-0.00518008 0.0131836 0.9999 -0.00166972],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CUDA:0, 0x130d2eae80]
[ 1.67625914e+12 -7.62900046e-04  2.01226748e-03 -1.66972354e-03
  7.55398050e-01  2.96797987e-01 -6.19902069e-02]


In [14]:
d.timestamp

1676259139169.7568

In [16]:
data[-1][0]-d.timestamp

0.0

In [22]:
r = mat_arr[-1]
r.inv()

[[0.999986 -0.00108192 -0.00518008 0.000756417],
 [0.00115013 0.999912 0.0131836 -0.00198920],
 [0.00516536 -0.0131894 0.9999 0.00170004],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={1, 4}, Float64, CUDA:0, 0x130d2a5400]

In [23]:
[Tracker.compute_translation(r.inv()), Tracker.compute_rotation(r.inv())]

[array([ 0.00075642, -0.0019892 ,  0.00170004]),
 [-0.7557288040169983, -0.29595474091515783, 0.06589860678100169]]

In [24]:
im=[]
for i in mat_arr:
    im.append(i.inv())

In [43]:
mat = o3d.core.Tensor([[0.999995, 0.001802, 0.002629, -0.000425],
[-0.001867, 0.999685, 0.025034, -0.003853],
[-0.002583, -0.025038, 0.999683, 0.001829],
[0.000000, 0.000000, 0.000000, 1.000000]])

In [44]:
[Tracker.compute_translation(mat), Tracker.compute_rotation(mat)]

Matrix is not a rotation matrix <ipykernel.iostream.OutStream object at 0x000001E9B75AD3F0>


SystemExit: 

In [40]:
mat

[[0.999995 0.00180200 0.00262900 -0.000425000],
 [-0.00186700 0.999685 0.0250340 -0.00385300],
 [-0.00258300 -0.0250380 0.999683 0.00182900],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CUDA:0, 0x1310206800]

In [42]:
%tb

SystemExit: 

In [45]:
 R = mat.numpy()[0:3].T[0:3].T.copy()

In [46]:
R

array([[ 0.999995,  0.001802,  0.002629],
       [-0.001867,  0.999685,  0.025034],
       [-0.002583, -0.025038,  0.999683]])

In [48]:
np.linalg.norm(np.identity(3, dtype=R.dtype) - np.dot(R.T, R))< 1e-6

False

In [52]:
R.T-np.linalg.inv(R)

array([[ 1.58233308e-07,  2.53557121e-07,  6.04436163e-08],
       [ 2.54767989e-07,  2.67069894e-07,  7.81686182e-07],
       [ 6.78551076e-08,  7.81242488e-07, -3.06547191e-07]])

In [53]:
np.dot(R.T, R)

array([[1.00000016e+00, 2.52249000e-07, 6.71880000e-08],
       [2.52249000e-07, 1.00000025e+00, 7.88794000e-07],
       [6.71880000e-08, 7.88794000e-07, 9.99999713e-01]])

In [55]:
np.linalg.norm(np.dot(R.T, R)-np.identity(3, dtype=R.dtype))

1.2446542359822753e-06

In [57]:
1-np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0])

3.257148304514068e-06

In [58]:
R

array([[ 0.999995,  0.001802,  0.002629],
       [-0.001867,  0.999685,  0.025034],
       [-0.002583, -0.025038,  0.999683]])

In [61]:
[Tracker.compute_translation(im[-20]), Tracker.compute_rotation(im[-20])]

[array([ 0.00112075, -0.0018567 ,  0.00171994]),
 [-0.7094151181259865, -0.41827439753900186, 0.021067586151859812]]

In [68]:
(data[-20][0]-data[0][0])/1000

693.7244582519531

In [70]:
d2=[]
for i,mat in enumerate(im):
    d2.append(np.concatenate(([data[i][0]], Tracker.compute_translation(im[i]), Tracker.compute_rotation(im[i]))))
np.savetxt('track.txt',d2)

In [71]:
np.savetxt('track.txt',d2)

In [80]:
o3d.io.write_point_cloud('600secs.pcd', d.compute_pcd(True).to_legacy())

True